In [1]:
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime
import base64

In [2]:
header={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36',
    'Referer':'https://www.google.com.tw/'
}

In [3]:
#https://free-proxy-list.net/
"""
proxy中，有可用與不可用的，成功率約20-25%，會有一大串無法成功的proxy
"""

def get_proxy():
    a=requests.get('https://free-proxy-list.net/',headers=header,timeout=20)
    soup=BeautifulSoup(a.text,'lxml')

    col_name=[s.text for s in soup.find(class_='table-responsive').find('tr').find_all('th')]

    d=[]

    for row in soup.find(class_='table-responsive').find_all('tr')[1:]:
        d.append([col.text for col in row])

    proxy_df=pd.DataFrame(d,columns=col_name)
    
    proxy_df=proxy_df[proxy_df.Anonymity.isin(['elite proxy','transparent'])]
    
    return proxy_df

In [4]:
#https://geonode.com/free-proxy-list/ 
"""
無任一proxy成功，表現非常差勁，不考慮
"""

def get_proxy_2():
    b=requests.get('https://proxylist.geonode.com/api/proxy-list?limit=200&sort_by=lastChecked&sort_type=desc&speed=fast&anonymityLevel=elite&anonymityLevel=anonymous').json()
    
    b=pd.DataFrame(b['data'])
    
    b=b.loc[:,['ip','port','country','org','asn','protocols','anonymityLevel','speed','upTime','responseTime','latency']]
    
    b=b[b['speed']<1000]
    
    b=b.rename(columns={
        'ip':'IP Address',
        'port':'Port'
    })
    
    return b

In [5]:
#https://hidemy.name/en/proxy-list/
"""
成功率5%，可參考
"""

def get_proxy_3():
    a=requests.get('https://hidemy.name/en/proxy-list/',headers=header)
    soup=BeautifulSoup(a.text,'lxml')    
    
    u=['https://hidemy.name'+s.get('href') for s in soup.find(class_='pagination').find_all('a') if s.get('href') is not None]
    
    u.remove('https://hidemy.name/en/proxy-list/#list')
    
    data=[]
    
    data.append(pd.read_html(str(soup.find(class_='table_block')))[0])
    
    for url in u:
        a=requests.get(url,headers=header)
        soup=BeautifulSoup(a.text,'lxml')      
        
        data.append(pd.read_html(str(soup.find(class_='table_block')))[0])
    
    data=pd.concat(data)
    
    data=data.rename(columns={'IP address':'IP Address'})
    
    return data[data.Anonymity != 'no']

In [6]:
#https://www.proxyscan.io/
"""
非常微小IP可成功(0.5%成功率)
"""

def get_proxy_4():
    #選500筆
    a=requests.get('https://www.proxyscan.io/home/filterresult?limit=500')
    soup=BeautifulSoup(a.text,'lxml')

    row=[]

    for r in soup.find_all('tr'):
        row.append([s.text.strip() for s in r.find_all(re.compile('td|th'))])

    data=pd.DataFrame(row,columns=[
        'IP Address','Port','Country,City','Ping','Type','Anonymity','Uptime','Last Status','Last Check'
    ])

    data=data[data['Last Status'] == 'OK']
    
    return data

In [7]:
#https://spys.one/en/free-proxy-list/
from functools import partial, reduce
from itertools import chain
from collections import Counter
from typing import Iterator

#找出多個string，包含最長的共同substring
def ngram(seq: str, n: int) -> Iterator[str]:
    return (seq[i: i+n] for i in range(0, len(seq)-n+1))

def allngram(seq: str) -> set:
    lengths = range(len(seq))
    ngrams = map(partial(ngram, seq), lengths)
    return set(chain.from_iterable(ngrams))

"""
要獲得正確的port，需要猜出他隨機產出的random number，並取代掉

測試成功率有10%
"""
def get_proxy_5():
    data=[]

    for iii in range(1,6):
        a=requests.get(f'http://nntime.com/proxy-list-0{iii}.htm',headers=header)
        soup=BeautifulSoup(a.text,'lxml')

        table=soup.find('table',{'id':'proxylist'})

        column_name=[s.text for s in table.find_all('tr')[1].find_all('th')]

        dat=[]

        first_guess_random=[part.find('td').find('input').get('value') for part in table.find_all('tr')[2:]]


        first_guess_random=[s.split('.')[-1] for s in first_guess_random]

        sequences = first_guess_random

        seqs_ngrams = map(allngram, sequences)
        counts = Counter(chain.from_iterable(seqs_ngrams))

        temp=pd.DataFrame(counts,index=[0]).T.reset_index(drop=False)
        temp=temp.sort_values([0],ascending=False)
        temp=temp[temp[0] == temp[0].iloc[0]]

        t=temp['index'].astype('str').map(len)

        replace_str=temp[t == t.max()]['index'].iloc[0]
        
        
        for part in table.find_all('tr')[2:]:
            row=[]

            for i,sub_part in enumerate(part.find_all('td')):
                if i == 0:
                    row.append(sub_part.find('input').get('value'))
                else:
                    row.append(sub_part.text.strip())

            ip=row[1]
            row[1]=ip[:ip.find('document')]

            port=re.search(string=row[0],pattern=f'{replace_str}(\d+)$').group(1)

            row[0]=port

            dat.append(row)

        dat=pd.DataFrame(dat,columns=column_name)

        dat=dat.rename(columns={'':'Port'})
        
        data.append(dat)

    data=pd.concat(data)

    data=data[~data.duplicated(subset=['IP Address'])].reset_index(drop=True)
    
    return data.loc[:,['IP Address','Port', 'Anonymity Type', 'Updated',
       'Country/State (City)', 'Registered to']]

## Test free proxy website can use or not

In [8]:
for fun in [get_proxy,get_proxy_2,get_proxy_3,get_proxy_4,get_proxy_5]:
    proxy_df=fun()

    proxy_list=[{'https':f'{i}:{j}','http':f'{i}:{j}'} for i,j in zip(proxy_df['IP Address'],proxy_df['Port'])] 
    
    url='https://ip.xxoo.net/?L=tw'
    
    print(f'Now test function is {fun}')
    
    N=0

    for now_proxy in proxy_list:
        try:
            a=requests.get(url,timeout=20,proxies=now_proxy)
            soup=BeautifulSoup(a.text,'lxml') 
            N=N+1
        except Exception as e:
            continue

    total_n=len(proxy_list)
    print(f'Total test is {total_n} , success is {N}.Percentage of success is {np.round(N/total_n,4)*100}')

Now test function is <function get_proxy at 0x000001687ADD94C8>
Total test is 219 , success is 42.Percentage of success is 19.18
Now test function is <function get_proxy_2 at 0x000001687ADD9A68>
Total test is 62 , success is 0.Percentage of success is 0.0
Now test function is <function get_proxy_3 at 0x00000168684784C8>
Total test is 74 , success is 4.Percentage of success is 5.41
Now test function is <function get_proxy_4 at 0x0000016867519A68>
Total test is 138 , success is 0.Percentage of success is 0.0
Now test function is <function get_proxy_5 at 0x0000016868538B88>
Total test is 174 , success is 18.Percentage of success is 10.34


In [9]:
proxy_df

,IP Address,Port,Anonymity Type,Updated,Country/State (City),Registered to
0,149.54.11.66,80,anonymous,Feb-08-2022,Afghanistan,BlueSky
1,61.29.96.146,80,anonymous proxy,Feb-08-2022,Australia (Melbourne),AAPT Limited
2,175.144.112.236,80,anonymous,Feb-08-2022,Malaysia (Kajang),TM Net
3,175.144.112.239,80,anonymous,Feb-08-2022,Malaysia (Kajang),TM Net
4,41.76.155.26,80,anonymous,Feb-08-2022,Nigeria (Lagos),SWIFTTALK Ltd
...,...,...,...,...,...,...
169,41.204.87.90,8080,transparent proxy,Feb-09-2022,Cameroon,Data communication and international
170,102.244.192.150,8080,transparent,Feb-08-2022,Cameroon,
171,41.204.93.54,8080,transparent proxy,Feb-08-2022,Cameroon,Cameroon Telecommunications Network
172,66.70.198.229,999,transparent,Feb-09-2022,Canada,"OLM, LLC"
